In [ ]:
from ultralytics import YOLO

# Load the YOLO12 model

import torch
import torchvision

model = YOLO('/Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/last_model_train12052025.pt')
model.predict(source="/Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/dashcamera/train/images/0c7fad6a-2dfd-4570-ba63-85b3e1ebb852.jpeg",save_txt=True,save =True,show=True)




image 1/1 /Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/dashcamera/train/images/0c7fad6a-2dfd-4570-ba63-85b3e1ebb852.jpeg: 320x512 1 Lane Blur, 511.1ms
Speed: 7.6ms preprocess, 511.1ms inference, 15.7ms postprocess per image at shape (1, 3, 320, 512)
Results saved to runs/detect/predict21
1 label saved to runs/detect/predict21/labels


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'Alligator Crack', 1: 'Block Crack', 2: 'Crosswalk Blur', 3: 'Lane Blur', 4: 'Longitudinal Crack', 5: 'Manhole', 6: 'Patch Repair', 7: 'Pothole', 8: 'Transverse Crack', 9: 'Wheel Mark Crack'}
 obb: None
 orig_img: array([[[ 93, 134, 127],
         [ 83, 121, 115],
         [ 96, 132, 126],
         ...,
         [ 80,  89, 102],
         [ 89,  98, 111],
         [105, 114, 127]],
 
        [[ 57,  98,  91],
         [ 41,  79,  73],
         [ 37,  73,  67],
         ...,
         [101, 113, 125],
         [ 52,  61,  74],
         [ 47,  59,  71]],
 
        [[ 66, 107, 100],
         [ 43,  81,  75],
         [ 42,  78,  72],
         ...,
         [171, 186, 195],
         [117, 129, 139],
         [ 54,  69,  78]],
 
        ...,
 
        [[108, 103, 104],
         [104,  99, 100],
         [103,  98,  99],
         ...,
         

: 

In [9]:
import os
import glob
import random
import shutil

# Parameters
base_dir = '/Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/אימון תמונות מהאופנוע'  # directory containing 'images' and 'labels' subdirs
output_dir = '/Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/dashcamera'
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1
random_seed = 42

# Paths
images_dir = os.path.join(base_dir, 'images')
labels_dir = os.path.join(base_dir, 'labels')

# Gather all image files
image_extensions = ('*.jpg', '*.jpeg', '*.png')
images = []
for ext in image_extensions:
    images.extend(glob.glob(os.path.join(images_dir, ext)))

# Shuffle for randomness
random.seed(random_seed)
random.shuffle(images)

# Compute split boundaries
total = len(images)
train_end = int(total * train_ratio)
val_end = train_end + int(total * val_ratio)

splits = {
    'train': images[:train_end],
    'val': images[train_end:val_end],
    'test': images[val_end:],
}

# Create and populate split directories
for split, img_paths in splits.items():
    img_out = os.path.join(output_dir, 'images', split)
    lbl_out = os.path.join(output_dir, 'labels', split)
    os.makedirs(img_out, exist_ok=True)
    os.makedirs(lbl_out, exist_ok=True)
    
    for img_path in img_paths:
        fname = os.path.basename(img_path)
        name, _ = os.path.splitext(fname)
        lbl_path = os.path.join(labels_dir, f'{name}.txt')
        
        # Copy image
        shutil.copy(img_path, os.path.join(img_out, fname))
        
        # Copy label if exists
        if os.path.exists(lbl_path):
            shutil.copy(lbl_path, os.path.join(lbl_out, f'{name}.txt'))



In [3]:
import os
import glob

# 👉 תיקיית המקור של ה-patches
PATCH_BASE = '/Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/patchs-2'

# 👉 תיקיית ה-יעד – רק labels יועתקו אליה
TARGET_BASE = '/Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/dashcamera'

# מיפוי תיקיות לפי split
splits = {
    'train': 'train/labels',
    'valid': 'valid/labels',
    'test':  'test/labels',
}

OLD_CLASS = '0'
NEW_CLASS = '6'

for split, lbl_sub in splits.items():
    src_lbl_dir = os.path.join(PATCH_BASE, lbl_sub)
    dst_lbl_dir = os.path.join(TARGET_BASE, lbl_sub)
    os.makedirs(dst_lbl_dir, exist_ok=True)

    if not os.path.isdir(src_lbl_dir):
        print(f"⚠️ לא נמצאה תיקיית תוויות עבור {split}: {src_lbl_dir}")
        continue

    for lbl_path in glob.glob(os.path.join(src_lbl_dir, '*.txt')):
        base = os.path.basename(lbl_path)
        dst_lbl_path = os.path.join(dst_lbl_dir, base)

        # אם הקובץ כבר קיים בתיקיית היעד, מוחקים אותו כדי להחליף
        if os.path.exists(dst_lbl_path):
            os.remove(dst_lbl_path)
            print(f"🗑️  {split}: {base} נמחק כדי להחליף בגרסה חדשה")

        # קוראים מהמקור, ממירים את class_id אם צריך, וכותבים ליעד
        with open(lbl_path, 'r') as rf, open(dst_lbl_path, 'w') as wf:
            for line in rf:
                parts = line.strip().split()
                if parts and parts[0] == OLD_CLASS:
                    parts[0] = NEW_CLASS
                wf.write(' '.join(parts) + '\n')

        print(f"🔄 {split}: {base} הועתק/עודכן בתיקייה {dst_lbl_dir}")

print("✨ סיום: כל התיוגים ממיפוי PATCH_BASE → TARGET_BASE הועתקו/הוחלפו עם class_id=0→6")


🗑️  train: WhatsApp-Image-2022-06-27-at-9-19-30-AM-2-_jpeg.rf.998a84a6b67bdc8b74d73c415f577360.txt נמחק כדי להחליף בגרסה חדשה
🔄 train: WhatsApp-Image-2022-06-27-at-9-19-30-AM-2-_jpeg.rf.998a84a6b67bdc8b74d73c415f577360.txt הועתק/עודכן בתיקייה /Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/dashcamera/train/labels
🗑️  train: var68_png.rf.d68771812ab31d565a2baa9143cc94aa.txt נמחק כדי להחליף בגרסה חדשה
🔄 train: var68_png.rf.d68771812ab31d565a2baa9143cc94aa.txt הועתק/עודכן בתיקייה /Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/dashcamera/train/labels
🗑️  train: WhatsApp-Image-2022-06-28-at-10-41-32-PM-1-_jpeg.rf.3512f2eea45b0ac26fcf14e069c5ddba.txt נמחק כדי להחליף בגרסה חדשה
🔄 train: WhatsApp-Image-2022-06-28-at-10-41-32-PM-1-_jpeg.rf.3512f2eea45b0ac26fcf14e069c5ddba.txt הועתק/עודכן בתיקייה /Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/dashcamera/train/labels
🗑️  train: WhatsApp-Image-2022-06-17-at

In [1]:
from ultralytics import YOLO

model = YOLO("/Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/last_model_train12052025.pt")

# 1. מספר מחלקות
print("🔢 Number of classes (nc):", model.model.nc)

# 2. שמות המחלקות
print("🏷️ Class names:", model.names)

# 3. אפשר לשחזר קובץ data.yaml בעזרת המידע הזה:
with open("reconstructed_data.yaml", "w") as f:
    f.write("path: /Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/dashcamera\n")  # עדכן אם יש צורך\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    f.write("test: images/test\n")
    f.write(f"nc: {model.model.nc}\n")
    f.write("names:\n")
    for name in model.names.values():
        f.write(f"  - {name}\n")


🔢 Number of classes (nc): 10
🏷️ Class names: {0: 'Alligator Crack', 1: 'Block Crack', 2: 'Crosswalk Blur', 3: 'Lane Blur', 4: 'Longitudinal Crack', 5: 'Manhole', 6: 'Patch Repair', 7: 'Pothole', 8: 'Transverse Crack', 9: 'Wheel Mark Crack'}


## הורדת ההדאטה של המודל 
# RDD2020 Dataset

In [2]:

from roboflow import Roboflow
rf = Roboflow(api_key="BQvvxKrJPoAaNkv4ZXlr")
project = rf.workspace("new-workspace-07lef").project("patchs")
version = project.version(2)
dataset = version.download("yolov12")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to patchs-2 in yolov12:: 100%|██████████| 3198/3198 [00:02<00:00, 1148.65it/s]
